In [1]:
# import urllib.request

# url = 'https://shapenet.cs.stanford.edu/ericyi/shapenetcore_partanno_v0.zip'
# urllib.request.urlretrieve(url, 'data.zip')
# from zipfile import ZipFile
# with ZipFile('data.zip', 'r') as zipObj:
#    # Extract all the contents of zip file in different directory
#    zipObj.extractall('data')
    
# url = 'https://github.com/chrdiller/pyTorchChamferDistance/archive/master.zip'
# urllib.request.urlretrieve(url, 'chamfer.zip')
# with ZipFile('chamfer.zip', 'r') as zipObj:
#    # Extract all the contents of zip file in different directory
#    zipObj.extractall('')
    
# url = 'https://github.com/meder411/PyTorch-EMDLoss/archive/master.zip'
# urllib.request.urlretrieve(url, 'emd.zip')
# with ZipFile('emd.zip', 'r') as zipObj:
#    # Extract all the contents of zip file in different directory
#    zipObj.extractall('')
    
# !python setup.py install

In [5]:
from __future__ import print_function
import argparse
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
from datasets import PartDataset
import torch.nn.functional as F
import torch.cuda as cuda
from pic2points_model import pic2points
from torch.nn.parallel import DataParallel
from torch.autograd import Variable
import torch
from chamfer_distance import ChamferDistance
from data_loader import XDataset, get_loader
#from emd import EMDLoss

#dist =  EMDLoss()

chamferDist = ChamferDistance()

# def main():
manualSeed = random.randint(1, 10000) # fix seed
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

#     dataset = PartDataset(root = 'data/PartAnnotation/', pic2point = True, npoints = 2500)
#     dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True, num_workers=8)
#     print("number of training data:"+ str(len(dataset)))
#     test_dataset = PartDataset(root = 'data/PartAnnotation/', pic2point = True, train = False, npoints = 2500)
#     testdataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16,shuffle=True, num_workers=8)
#     print("number of testing data:"+ str(len(test_dataset)))


# Decide on GPU or CPU
if torch.cuda.is_available():
    gpu_or_cpu = torch.device('cuda')
else:
    gpu_or_cpu = torch.device('cpu')

# Training Configuration
image_root = "./../../../datasets/cs253-wi20-public/ShapeNetRendering/"
point_cloud_root = "./../../../datasets/cs253-wi20-public/ShapeNet_pointclouds/"

num_epochs = 20
batch_size = 4
shuffle = True
num_workers = 8
use_2048 = True
img_size = 227
learning_rate = 1e-3
num_points = 2500
transform = transforms.Compose([transforms.Resize(img_size,interpolation=2),transforms.CenterCrop(img_size)])

# Data loader
data_loader = get_loader(image_root, point_cloud_root, use_2048, transform, batch_size, shuffle, num_workers)

# create model
print("model building...")
model = pic2points(num_points=num_points)
model.to(device=gpu_or_cpu)

# load pre-existing weights
#  if opt.model != '':
#     model.load_state_dict(torch.load(opt.model))

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     num_batch = len(dataset) / batch_size

print('training mode ------------------')
    
# if __name__ == '__main__':
#     num_cuda = cuda.device_count()
#     print("number of GPUs have been detected:" + str(num_cuda))
#     #with torch.cuda.device(1):
#     main()

Random Seed:  3907
model building...
training mode ------------------


In [15]:
# print(model)

In [14]:
len(data_loader)

262704

In [13]:
for epoch in range(num_epochs):
    print("epoch:"+str(epoch))
    for i, (image, point_cloud) in enumerate(data_loader):
#         print('image type = ', type(image))
#         print('image size = ', image.size())
#         print('point_cloud size = ', point_cloud.size())
        image, point_cloud = Variable(image), Variable(point_cloud)
        
        image, point_cloud = image.float().to(device=gpu_or_cpu), point_cloud.float().to(device=gpu_or_cpu)
        pred = model(image)
        dist1, dist2 = chamferDist(pred, point_cloud)
        loss = (torch.mean(dist1)) + (torch.mean(dist2))
#             emd_cost = torch.sum(dist(pred.cuda().double(), points.cuda().double()))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 10000 == 0:
            print("training loss, " + "Batch "+ str(i) + ": " + str(loss.item()))

#         loss_test = 0
#         for i, data in enumerate(dataloader, 0):
#             im_test, points_test = data
#             im_test, points_test = Variable(im_test), Variable(points_test)
#             im_test, points_test = im_test.cuda(), points_test.cuda()
#             pred_test = model(im_test)
#             dist1, dist2 = chamferDist(pred_test, points_test)
#             loss_test = (torch.mean(dist1)) + (torch.mean(dist2))
# #             emd_test = torch.sum(dist(pred_test.cuda().double(), points_test.cuda().double()))
#         print("Testing loss is:" + str(loss_test.item()))


epoch:0
training loss, Batch 0: 0.006113636773079634
training loss, Batch 50: 0.017141668125987053
training loss, Batch 100: 0.006510607432574034
training loss, Batch 150: 0.01705508679151535
training loss, Batch 200: 0.006557357497513294
training loss, Batch 250: 0.004852999933063984
training loss, Batch 300: 0.005551069043576717
training loss, Batch 350: 0.008703025057911873
training loss, Batch 400: 0.007491051685065031
training loss, Batch 450: 0.019877925515174866
training loss, Batch 500: 0.007612662389874458
training loss, Batch 550: 0.006605297327041626
training loss, Batch 600: 0.010538516566157341
training loss, Batch 650: 0.008715455420315266
training loss, Batch 700: 0.004682359751313925
training loss, Batch 750: 0.008188946172595024
training loss, Batch 800: 0.004377533681690693
training loss, Batch 850: 0.009123516269028187
training loss, Batch 900: 0.00956349540501833
training loss, Batch 950: 0.0069265482015907764
training loss, Batch 1000: 0.005137745290994644
training

training loss, Batch 8550: 0.004610544536262751
training loss, Batch 8600: 0.00423802062869072
training loss, Batch 8650: 0.006742898840457201
training loss, Batch 8700: 0.004402045160531998
training loss, Batch 8750: 0.005068531259894371
training loss, Batch 8800: 0.0030988899525254965
training loss, Batch 8850: 0.009968934580683708
training loss, Batch 8900: 0.0034082215279340744
training loss, Batch 8950: 0.006077205762267113
training loss, Batch 9000: 0.00435540871694684
training loss, Batch 9050: 0.0030963404569774866
training loss, Batch 9100: 0.004657597281038761
training loss, Batch 9150: 0.019309328868985176
training loss, Batch 9200: 0.004748796112835407
training loss, Batch 9250: 0.007040685974061489
training loss, Batch 9300: 0.006272414233535528
training loss, Batch 9350: 0.002766710240393877
training loss, Batch 9400: 0.005343594588339329
training loss, Batch 9450: 0.0037859634030610323
training loss, Batch 9500: 0.011338794603943825
training loss, Batch 9550: 0.003713092

training loss, Batch 16950: 0.0034919166937470436
training loss, Batch 17000: 0.005305989645421505
training loss, Batch 17050: 0.005321683827787638
training loss, Batch 17100: 0.005682835821062326
training loss, Batch 17150: 0.0056601837277412415
training loss, Batch 17200: 0.002602561842650175
training loss, Batch 17250: 0.005928915925323963
training loss, Batch 17300: 0.005809373687952757
training loss, Batch 17350: 0.0024105471093207598
training loss, Batch 17400: 0.009708695113658905
training loss, Batch 17450: 0.005236022174358368
training loss, Batch 17500: 0.0044760024175047874
training loss, Batch 17550: 0.003495407523587346
training loss, Batch 17600: 0.006960595957934856
training loss, Batch 17650: 0.009940465912222862
training loss, Batch 17700: 0.0034862940665334463
training loss, Batch 17750: 0.005680235102772713
training loss, Batch 17800: 0.004447353072464466
training loss, Batch 17850: 0.0062599778175354
training loss, Batch 17900: 0.010235847905278206
training loss, Ba

training loss, Batch 25300: 0.00604806374758482
training loss, Batch 25350: 0.003679070621728897
training loss, Batch 25400: 0.005833845119923353
training loss, Batch 25450: 0.0037771929055452347
training loss, Batch 25500: 0.007679418660700321
training loss, Batch 25550: 0.005182782653719187
training loss, Batch 25600: 0.002079431200399995
training loss, Batch 25650: 0.003146043512970209
training loss, Batch 25700: 0.006759571842849255
training loss, Batch 25750: 0.0049604088999331
training loss, Batch 25800: 0.004399582743644714
training loss, Batch 25850: 0.00351519463583827
training loss, Batch 25900: 0.0054592955857515335
training loss, Batch 25950: 0.0033131628297269344
training loss, Batch 26000: 0.003932913765311241
training loss, Batch 26050: 0.004677544813603163
training loss, Batch 26100: 0.004942447412759066
training loss, Batch 26150: 0.0074350303038954735
training loss, Batch 26200: 0.0032895677722990513
training loss, Batch 26250: 0.0018793377093970776
training loss, Bat

training loss, Batch 33650: 0.0048675984144210815
training loss, Batch 33700: 0.0036408137530088425
training loss, Batch 33750: 0.004517735913395882
training loss, Batch 33800: 0.0032747462391853333
training loss, Batch 33850: 0.0035774498246610165
training loss, Batch 33900: 0.006899246014654636
training loss, Batch 33950: 0.003888916689902544
training loss, Batch 34000: 0.010050100274384022
training loss, Batch 34050: 0.0027087097987532616
training loss, Batch 34100: 0.00901820883154869
training loss, Batch 34150: 0.004557190928608179
training loss, Batch 34200: 0.006500198505818844
training loss, Batch 34250: 0.005741229746490717
training loss, Batch 34300: 0.004154632333666086
training loss, Batch 34350: 0.0035498901270329952
training loss, Batch 34400: 0.0020682201720774174
training loss, Batch 34450: 0.009524148888885975
training loss, Batch 34500: 0.004761679098010063
training loss, Batch 34550: 0.005918489769101143
training loss, Batch 34600: 0.006431677378714085
training loss,

...

training loss, Batch 108650: 0.004341698717325926
training loss, Batch 108700: 0.0031434278935194016
training loss, Batch 108750: 0.002682336140424013
training loss, Batch 108800: 0.004618508275598288
training loss, Batch 108850: 0.004493852145969868
training loss, Batch 108900: 0.005038408562541008
training loss, Batch 108950: 0.007254820317029953
training loss, Batch 109000: 0.00395355187356472
training loss, Batch 109050: 0.0017991713248193264
training loss, Batch 109100: 0.004068957641720772
training loss, Batch 109150: 0.003095801919698715
training loss, Batch 109200: 0.0043366458266973495
training loss, Batch 109250: 0.0010147604625672102
training loss, Batch 109300: 0.0035622476134449244
training loss, Batch 109350: 0.007113432511687279
training loss, Batch 109400: 0.0064117396250367165
training loss, Batch 109450: 0.005898171104490757
training loss, Batch 109500: 0.0016516605392098427
training loss, Batch 109550: 0.0030347006395459175
training loss, Batch 109600: 0.004048400558

training loss, Batch 116800: 0.004119118209928274
training loss, Batch 116850: 0.004790591541677713
training loss, Batch 116900: 0.004837149754166603
training loss, Batch 116950: 0.002969900844618678
training loss, Batch 117000: 0.009363319724798203
training loss, Batch 117050: 0.006094101816415787
training loss, Batch 117100: 0.0023488420993089676
training loss, Batch 117150: 0.003470875322818756
training loss, Batch 117200: 0.003021545009687543
training loss, Batch 117250: 0.0022014339920133352
training loss, Batch 117300: 0.004231839440762997
training loss, Batch 117350: 0.0037938510067760944
training loss, Batch 117400: 0.005695745348930359
training loss, Batch 117450: 0.003330992069095373
training loss, Batch 117500: 0.002752000465989113
training loss, Batch 117550: 0.003675345331430435
training loss, Batch 117600: 0.003715165425091982
training loss, Batch 117650: 0.006277009844779968
training loss, Batch 117700: 0.002422286896035075
training loss, Batch 117750: 0.0037026382051408

training loss, Batch 125000: 0.00745764933526516
training loss, Batch 125050: 0.003967026714235544
training loss, Batch 125100: 0.005786390975117683
training loss, Batch 125150: 0.005474704317748547
training loss, Batch 125200: 0.003311881795525551
training loss, Batch 125250: 0.005063138902187347
training loss, Batch 125300: 0.0016500280471518636
training loss, Batch 125350: 0.008798671886324883
training loss, Batch 125400: 0.004916992038488388
training loss, Batch 125450: 0.005296687595546246
training loss, Batch 125500: 0.006838257424533367
training loss, Batch 125550: 0.017351225018501282
training loss, Batch 125600: 0.0020284936763346195
training loss, Batch 125650: 0.003549945540726185
training loss, Batch 125700: 0.002029519295319915
training loss, Batch 125750: 0.007517099846154451
training loss, Batch 125800: 0.0028773928061127663
training loss, Batch 125850: 0.0019260330591350794
training loss, Batch 125900: 0.0030147205106914043
training loss, Batch 125950: 0.003865388454869

training loss, Batch 133150: 0.004235154017806053
training loss, Batch 133200: 0.0012352983467280865
training loss, Batch 133250: 0.008053064346313477
training loss, Batch 133300: 0.0029267838690429926
training loss, Batch 133350: 0.008915454149246216
training loss, Batch 133400: 0.005730926990509033
training loss, Batch 133450: 0.004965703934431076
training loss, Batch 133500: 0.0036684460937976837
training loss, Batch 133550: 0.004348812624812126
training loss, Batch 133600: 0.004658529534935951
training loss, Batch 133650: 0.0034641376696527004
training loss, Batch 133700: 0.0036244881339371204
training loss, Batch 133750: 0.002542935311794281
training loss, Batch 133800: 0.003944899886846542
training loss, Batch 133850: 0.002842795569449663
training loss, Batch 133900: 0.0035922816023230553
training loss, Batch 133950: 0.00391179695725441
training loss, Batch 134000: 0.0027458653785288334
training loss, Batch 134050: 0.0034018829464912415
training loss, Batch 134100: 0.006756570655

KeyboardInterrupt: 

In [ ]:
import torchvision.transforms as transforms

normalize = transforms.Compose([transforms.ToTensor()])

x = torch.randn((256, 256, 3))

In [ ]:
y = np.asarray(x)

In [ ]:
z = normalize(y)

In [ ]:
z.size()